In [1]:
# 统一设置 NOTEBOOK_DIR（本文件所在目录）
from pathlib import Path
import os
NOTEBOOK_DIR = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd().resolve()
print('NOTEBOOK_DIR =', NOTEBOOK_DIR)

NOTEBOOK_DIR = D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通


## Case1 - Excel Merge (员工绩效合并)

In [ ]:
# 如果需要，先切换工作目录到 Case1 文件夹
import os, pandas as pd
BASE = NOTEBOOK_DIR / 'Case1_Excel_merge'
BASE.mkdir(exist_ok=True)
os.chdir(BASE)

# 读取 Excel
basic_info_df = pd.read_excel('员工基本信息表.xlsx')
performance_df = pd.read_excel('员工绩效表.xlsx')

# 过滤 2024 Q4
q4_2024_perf = performance_df[(performance_df['年度'] == 2024) & (performance_df['季度'] == 4)]

# 合并
merged_df = pd.merge(basic_info_df, q4_2024_perf[['员工ID', '绩效评分']], on='员工ID', how='left')
merged_df = merged_df.rename(columns={'绩效评分': '2024年第四季度绩效评分'})

print(merged_df.head())
# 如需保存： merged_df.to_excel('员工基本信息及2024年第四季度绩效评分.xlsx', index=False)

## Case2 - 医院病床使用可视化 (Flask + ECharts)
按顺序跑下面几个单元格，边跑边看日志/数据，就能理解流程，不用把 .py 复制进来。

In [2]:
# Step 0: 安装依赖（首次/缺包时运行）
# 更新了 requirements.txt，已包含 openpyxl
# %pip install -r requirements.txt

In [3]:
# !python 1_precompute_data.py
# !python 2_view_excel_data.py
# !python 3_app.py

In [4]:
# Step 1: 快速查看原始 Excel 列和前几行
import pandas as pd, os
BASE = NOTEBOOK_DIR / 'Case2_bed_usage'
os.chdir(BASE)
df = pd.read_excel('hospital_bed_usage_data.xlsx')
print('读取自:', BASE / 'hospital_bed_usage_data.xlsx')
print('列名:', list(df.columns))
display(df.head())

读取自: D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\hospital_bed_usage_data.xlsx
列名: ['hospital_id', 'hospital_name', 'hospital_district', 'department_id', 'department_name', 'ward_id', 'ward_name', 'total_beds', 'occupied_beds', 'available_beds', 'occupancy_rate', 'timestamp', 'special_status']


,hospital_id,hospital_name,hospital_district,department_id,department_name,ward_id,ward_name,total_beds,occupied_beds,available_beds,occupancy_rate,timestamp,special_status
0,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W1,内科普通病房1号,23,23,0,99.00,2023-12-01,满床
1,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W2,内科普通病房2号,20,20,0,99.00,2023-12-01,满床
2,HH001,玛丽医院,港岛,DEPT01,内科,HH001_DEPT01_W3,内科普通病房3号,28,28,0,99.00,2023-12-01,满床
3,HH001,玛丽医院,港岛,DEPT02,外科,HH001_DEPT02_W1,外科特需病房1号,6,6,0,94.69,2023-12-01,正常
4,HH001,玛丽医院,港岛,DEPT03,儿科,HH001_DEPT03_W1,儿科普通病房1号,37,35,2,94.81,2023-12-01,正常


In [5]:
# Step 2: 预计算并写入 data_cache/
%run 1_precompute_data.py

2026-01-13 15:38:50,283 - __main__ - INFO - 开始预计算数据...
2026-01-13 15:38:50,295 - __main__ - INFO - 读取Excel文件: D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\hospital_bed_usage_data.xlsx
2026-01-13 15:38:54,370 - __main__ - INFO - Excel文件读取完成，共 59773 行数据
2026-01-13 15:38:54,371 - __main__ - INFO - 计算医院使用率数据...
2026-01-13 15:38:54,381 - __main__ - INFO - 计算科室使用率数据...
2026-01-13 15:38:54,387 - __main__ - INFO - 计算概览数据...
2026-01-13 15:38:54,392 - __main__ - INFO - 计算热力图数据...
2026-01-13 15:38:54,411 - __main__ - INFO - 缓存数据已保存到 D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\data_cache\data_cache.pkl
2026-01-13 15:38:54,416 - __main__ - INFO - 元数据已保存到 D:\Job_hunt\2025Courses_LLM\3-Cursor编程-从入门到精通\Case2_bed_usage\data_cache\metadata.json
2026-01-13 15:38:54,417 - __main__ - INFO - 数据预计算完成，耗时: 4.13 秒


数据预计算完成，请运行 app.py 查看结果


In [6]:
# Step 3: 查看缓存结构
import pickle, json

cache = pickle.load(open('data_cache/data_cache.pkl', 'rb'))
print('缓存键:', list(cache.keys()))
print('summary_data 示例:', json.dumps(cache.get('summary_data', {}), ensure_ascii=False, indent=2))

缓存键: ['hospital_usage', 'department_usage', 'summary_data', 'heatmap_data', 'timestamp', 'excel_last_modified', 'excel_md5']
summary_data 示例: {
  "total_beds": 1443095,
  "occupied_beds": 1192292,
  "available_beds": 209855,
  "occupancy_rate": 82.62,
  "top_departments": {
    "names": [
      "眼科",
      "皮肤科",
      "耳鼻喉科",
      "泌尿科",
      "妇产科"
    ],
    "values": [
      27236,
      27179,
      22546,
      18316,
      17958
    ]
  }
}


In [7]:
# Step 4: 生成探索型图表（输出在 charts/）
# %run 2_view_excel_data.py

In [ ]:
# Step 5: 启动 Flask（会常驻，占用这个单元格）
# 推荐在外部终端运行：python 3_app.py --port 5000
# 如需在 notebook 中临时跑，执行后自动打开浏览器访问 http://127.0.0.1:5000
# 中断该单元格即可停止
import webbrowser, sys, subprocess, time

# 启动 Flask
proc = subprocess.Popen([sys.executable, "3_app.py", "--port", "5000"])

# 等待服务启动片刻，然后自动打开浏览器
time.sleep(2)
webbrowser.open("http://127.0.0.1:5000/")

# 阻塞当前单元格，直到你手动中断（Kernel -> Interrupt）
proc.wait()

## Case3 - 疫情数据可视化 (Flask)
数据源：Case3_dashboard_epidemic/香港各区疫情数据_20250322.xlsx

In [ ]:
# Step 0: 安装依赖（如未安装）
import os
BASE = NOTEBOOK_DIR / 'Case3_dashboard_epidemic'
os.chdir(BASE)
# %pip install -r requirements.txt

In [ ]:
# Step 1: 可选预览原始数据结构
import pandas as pd
df = pd.read_excel('香港各区疫情数据_20250322.xlsx')
print(df.head())
print('列名:', list(df.columns))

In [ ]:
# Step 2: 启动 Case3 Flask（占用单元格，需手动中断停止）
import webbrowser, sys, subprocess, time
proc = subprocess.Popen([sys.executable, 'app.py', '--port', '5000'])
time.sleep(2)
webbrowser.open('http://127.0.0.1:5000/')
proc.wait()  # 中断单元格即可停止